In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
import zipfile
import cv2

## Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ROOT_DIR = '/content/gdrive/MyDrive/PCOS/train'
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
   number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))
   print("", dir, "", number_of_images[dir])

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Define a function for data augmentation and preprocessing
def preprocess_data(path, split):
    if not os.path.exists('/content/gdrive/MyDrive/PCOS/train' + path):
        os.mkdir("/content/gdrive/MyDrive/PCOS/train" + path)

        for dir in os.listdir(ROOT_DIR):
            os.makedirs("/content/gdrive/MyDrive/PCOS/train" + path + "/" + dir)
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR, dir)), size=(math.floor(split * number_of_images[dir]) - 5), replace=False):
                O = os.path.join(ROOT_DIR, dir, img)
                D = os.path.join("/content/gdrive/MyDrive/PCOS/train" + path, dir)
                shutil.copy(O, D)

    else:
        print("Folder already exists")

# Preprocess the data
preprocess_data('train', 0.7)
preprocess_data("val", 0.15)
preprocess_data("test", 0.15)

# Define functions for data loading and preprocessing
def preprocessingImage1(path):
    image_data = ImageDataGenerator(
        zoom_range=0.2,
        shear_range=0.2,
        preprocessing_function=preprocess_input,
        horizontal_flip=True
    )
    image = image_data.flow_from_directory(
        directory=path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )
    return image

def preprocessionfImage2(path):
    image_data = ImageDataGenerator(preprocessing_function=preprocess_input)
    image = image_data.flow_from_directory(
        directory=path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )
    return image

# Load and preprocess the data
path = '/content/gdrive/MyDrive/PCOS/train'
train_data = preprocessingImage1(path)

path = '/content/gdrive/MyDrive/PCOS/traintest'
test_data = preprocessionfImage2(path)

path = '/content/gdrive/MyDrive/PCOS/trainval'
val_data = preprocessionfImage2(path)

Mounted at /content/gdrive
 infected  781
 notinfected  1143
Folder already exists
Folder already exists
Folder already exists
Found 1924 images belonging to 2 classes.
Found 278 images belonging to 2 classes.
Found 278 images belonging to 2 classes.


## Model Building

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam

input_shape = (224, 224, 3)

# Define the input layer
inputs = Input(shape=input_shape)

# Convolutional blocks
x = Conv2D(32, (3, 3), activation='relu', strides=1, padding='valid')(inputs)
x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

# Convolutional block 2
x = Conv2D(32, (3, 3), activation='relu', strides=1, padding='valid')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

# Convolutional block 3
x = Conv2D(64, (3, 3), activation='relu', strides=1, padding='valid')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

# Convolutional block 4
x = Conv2D(64, (3, 3), activation='relu', strides=1, padding='valid')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

# Convolutional block 5
x = Conv2D(128, (3, 3), activation='relu', strides=1, padding='valid')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

# Flatten the output
x = Flatten()(x)

# Dense layers
x = Dense(124, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)

# Output layer for binary classification
output = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=inputs, outputs=output)

# Define callbacks
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5, verbose=1)
cb = [mc, es]

learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

model.summary()

model.fit(train_data, epochs=10, batch_size=16, validation_data=val_data)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 394272)            0         
                                                                 
 dense_3 (Dense)             (None, 124)               48889852  
                                                                 
 dropout_1 (Dropout)         (None, 124)               0         
                                                           

## Evaluating the Model

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

# Predict the labels for the test data
y_pred = model.predict(test_data)
y_pred = np.round(y_pred)

# Get the true labels
y_true = test_data.labels

# Display Classification Report and Confusion Matrix
print("Classification Report:")
print(classification_report(y_true, y_pred))
print("Confusion Matrix: ")
print(confusion_matrix(y_true, y_pred))


9/9 [==============================] - 84s 10s/step
Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.38      0.38       112
           1       0.58      0.59      0.59       166

    accuracy                           0.50       278
   macro avg       0.48      0.48      0.48       278
weighted avg       0.50      0.50      0.50       278

Confusion Matrix: 
[[42 70]
 [68 98]]
